In [2]:
import os
import shutil
from pathlib import Path

def extract_model_name(filename_or_dirname):
    """PDF 파일명이나 디렉토리명에서 모델명을 추출"""
    # .pdf 확장자 제거
    name = filename_or_dirname.replace('.pdf', '')

    # 일부 파일은 끝에 'T'나 다른 문자가 추가되어 있을 수 있으므로
    # 기본 모델명을 추출하기 위해 공통 패턴을 찾음
    return name

def normalize_model_name(name):
    """모델명을 정규화하여 비교하기 쉽게 만듦"""
    # 공백, 대소문자 등을 정규화
    normalized = name.strip().upper()

    # 일부 변형된 형태들을 처리
    # 예: DV21DG8200BVT -> DV21DG8200BV
    if normalized.endswith('T'):
        return normalized[:-1]

    return normalized

def find_matching_models():
    """PDF 파일과 이미지 폴더가 모두 있는 모델들을 찾기"""

    # 기본 경로 설정
    pdf_dir = Path("samsung_manuals")
    img_dir = Path("top3_all_color_all_image")

    # PDF 파일 목록 가져오기
    pdf_files = []
    if pdf_dir.exists():
        pdf_files = [f.name for f in pdf_dir.glob("*.pdf")]

    # 이미지 폴더 목록 가져오기
    img_folders = []
    if img_dir.exists():
        img_folders = [f.name for f in img_dir.iterdir() if f.is_dir()]

    print(f"발견된 PDF 파일: {len(pdf_files)}개")
    print(f"발견된 이미지 폴더: {len(img_folders)}개")

    # 모델명 매칭
    matched_models = []

    for pdf_file in pdf_files:
        pdf_model = normalize_model_name(extract_model_name(pdf_file))

        for img_folder in img_folders:
            img_model = normalize_model_name(img_folder)

            # 정확한 매칭 또는 부분 매칭 확인
            if pdf_model == img_model or pdf_model in img_model or img_model in pdf_model:
                # 더 정확한 매칭을 위해 핵심 모델 번호 비교
                pdf_core = extract_core_model(pdf_model)
                img_core = extract_core_model(img_model)

                if pdf_core == img_core:
                    matched_models.append((pdf_file, img_folder))
                    print(f"매칭 발견: {pdf_file} <-> {img_folder}")
                    break

    return matched_models

def extract_core_model(model_name):
    """모델명에서 핵심 모델 번호 추출 (예: DV21DG8200BV)"""
    import re

    # 영문자와 숫자로 구성된 핵심 모델 번호 추출
    # 예: AI_건조기_21KG_DV21DG8200BVT -> DV21DG8200BV
    pattern = r'([A-Z]{2}\d+[A-Z]+\d+[A-Z]+)'
    match = re.search(pattern, model_name)

    if match:
        return match.group(1)

    # 패턴이 맞지 않으면 전체 모델명 반환
    return model_name

def create_classifier_output(matched_models):
    """매칭된 모델들을 새로운 폴더 구조로 복사"""

    # 출력 폴더 생성
    output_base = Path("classifier_output")
    pdf_output = output_base / "samsung_manuals_classifier_output"
    img_output = output_base / "samsung_img_classifier_output"

    # 폴더 생성
    pdf_output.mkdir(parents=True, exist_ok=True)
    img_output.mkdir(parents=True, exist_ok=True)

    print(f"\n출력 폴더 생성 완료:")
    print(f"- {pdf_output}")
    print(f"- {img_output}")

    # 파일 복사
    copied_count = 0

    for pdf_file, img_folder in matched_models:
        try:
            # PDF 파일 복사
            src_pdf = Path("samsung_manuals") / pdf_file
            dst_pdf = pdf_output / pdf_file

            if src_pdf.exists():
                shutil.copy2(src_pdf, dst_pdf)
                print(f"PDF 복사: {pdf_file}")

            # 이미지 폴더 복사
            src_img = Path("top3_all_color_all_image") / img_folder
            dst_img = img_output / img_folder

            if src_img.exists():
                shutil.copytree(src_img, dst_img, dirs_exist_ok=True)
                print(f"이미지 폴더 복사: {img_folder}")

            copied_count += 1

        except Exception as e:
            print(f"복사 중 오류 발생 ({pdf_file}, {img_folder}): {e}")

    print(f"\n총 {copied_count}개 모델의 데이터가 복사되었습니다.")

def main():
    """메인 실행 함수"""
    print("Samsung 세탁기/건조기 모델 분류기 시작")
    print("=" * 50)

    # 현재 디렉토리 확인
    current_dir = Path.cwd()
    print(f"현재 작업 디렉토리: {current_dir}")

    # 필요한 폴더가 존재하는지 확인
    required_dirs = ["samsung_manuals", "samsung_img"]
    missing_dirs = []

    for dir_name in required_dirs:
        if not Path(dir_name).exists():
            missing_dirs.append(dir_name)

    if missing_dirs:
        print(f"❌ 다음 폴더들이 없습니다: {', '.join(missing_dirs)}")
        print("필요한 폴더들을 확인하고 다시 실행해주세요.")
        return

    # 매칭되는 모델 찾기
    print("\n📋 PDF와 이미지가 모두 있는 모델 찾는 중...")
    matched_models = find_matching_models()

    if not matched_models:
        print("❌ 매칭되는 모델을 찾을 수 없습니다.")
        return

    print(f"\n✅ 총 {len(matched_models)}개의 매칭되는 모델을 발견했습니다.")

    # 사용자 확인
    print("\n매칭된 모델 목록:")
    for i, (pdf_file, img_folder) in enumerate(matched_models, 1):
        print(f"{i:2d}. PDF: {pdf_file}")
        print(f"    IMG: {img_folder}")

    response = input(f"\n이 {len(matched_models)}개 모델을 classifier_output 폴더로 복사하시겠습니까? (y/n): ")

    if response.lower() in ['y', 'yes', '예', 'ㅇ']:
        print("\n📁 파일 복사 시작...")
        create_classifier_output(matched_models)
        print("\n✅ 작업 완료!")
    else:
        print("❌ 작업이 취소되었습니다.")

if __name__ == "__main__":
    main()

Samsung 세탁기/건조기 모델 분류기 시작
현재 작업 디렉토리: C:\Workspaces\SKN14-3rd-3Team\webscraping\YooYonghwan\model_classifier

📋 PDF와 이미지가 모두 있는 모델 찾는 중...
발견된 PDF 파일: 71개
발견된 이미지 폴더: 93개
매칭 발견: AI_건조기_21kg_DV21DG8200BVT.pdf <-> AI_건조기_21kg_DV21DG8200BV
매칭 발견: AI_건조기_21kg_DV21DG8600BVT.pdf <-> AI_건조기_21kg_DV21DG8600BV
매칭 발견: AI_세탁기_12kg_WW12T504DTW.pdf <-> AI_세탁기_12kg_WW12T504DTW
매칭 발견: AI_세탁기_21kg_WF21DG6650BV.pdf <-> AI_세탁기_21kg_WF21DG6650BV
매칭 발견: AI_세탁기_25kg_WF25DG8250BV.pdf <-> AI_세탁기_25kg_WF25DG8250BV
매칭 발견: AI_세탁기_25kg_WF25DG8650BV.pdf <-> AI_세탁기_25kg_WF25DG8650BV
매칭 발견: AI_통버블_세탁기_19kg_WA80F19E8L.pdf <-> AI_통버블_세탁기_19kg_WA80F19E8L
매칭 발견: AI_통버블_세탁기_19kg_WA80F19S8W.pdf <-> AI_통버블_세탁기_19kg_WA80F19S8W
매칭 발견: AI_통버블_세탁기_19kg_WA80F19SKB.pdf <-> AI_통버블_세탁기_19kg_WA80F19SKB
매칭 발견: Bespoke_AI_건조기_슬림_10kg_DV10BB8440GH.pdf <-> Bespoke_AI_건조기_슬림_10kg_DV10BB8440GH
매칭 발견: Bespoke_AI_세탁기_21kg_WF21CB6650BW.pdf <-> Bespoke_AI_세탁기_21kg_WF21CB6650BW
매칭 발견: Bespoke_AI_세탁기_24kg_WF24CB8650BW.pdf <-> Bespoke_AI_세탁기_2

In [2]:
import os
import shutil
from pathlib import Path

def extract_model_name(filename_or_dirname):
    """PDF 파일명이나 디렉토리명에서 모델명을 추출"""
    # .pdf 확장자 제거
    name = filename_or_dirname.replace('.pdf', '')

    # 일부 파일은 끝에 'T'나 다른 문자가 추가되어 있을 수 있으므로
    # 기본 모델명을 추출하기 위해 공통 패턴을 찾음
    return name

def normalize_model_name(name):
    """모델명을 정규화하여 비교하기 쉽게 만듦"""
    # 공백, 대소문자 등을 정규화
    normalized = name.strip().upper()

    # 일부 변형된 형태들을 처리
    # 예: DV21DG8200BVT -> DV21DG8200BV
    if normalized.endswith('T'):
        return normalized[:-1]

    return normalized

def find_matching_models():
    """PDF 파일과 이미지 폴더가 모두 있는 모델들을 찾기"""

    # 기본 경로 설정
    pdf_dir = Path("../pdf_downloader/samsung_manuals")
    img_dir = Path("../image_downloader/image_downloader/samsung_img")

    # PDF 파일 목록 가져오기
    pdf_files = []
    if pdf_dir.exists():
        pdf_files = [f.name for f in pdf_dir.glob("*.pdf")]

    # 이미지 폴더 목록 가져오기
    img_folders = []
    if img_dir.exists():
        img_folders = [f.name for f in img_dir.iterdir() if f.is_dir()]

    print(f"발견된 PDF 파일: {len(pdf_files)}개")
    print(f"발견된 이미지 폴더: {len(img_folders)}개")

    # 모델명 매칭
    matched_models = []

    for pdf_file in pdf_files:
        pdf_model = normalize_model_name(extract_model_name(pdf_file))

        for img_folder in img_folders:
            img_model = normalize_model_name(img_folder)

            # 정확한 매칭 또는 부분 매칭 확인
            if pdf_model == img_model or pdf_model in img_model or img_model in pdf_model:
                # 더 정확한 매칭을 위해 핵심 모델 번호 비교
                pdf_core = extract_core_model(pdf_model)
                img_core = extract_core_model(img_model)

                if pdf_core == img_core:
                    matched_models.append((pdf_file, img_folder))
                    print(f"매칭 발견: {pdf_file} <-> {img_folder}")
                    break

    return matched_models

def extract_core_model(model_name):
    """모델명에서 핵심 모델 번호 추출 (예: DV21DG8200BV)"""
    import re

    # 영문자와 숫자로 구성된 핵심 모델 번호 추출
    # 예: AI_건조기_21KG_DV21DG8200BVT -> DV21DG8200BV
    pattern = r'([A-Z]{2}\d+[A-Z]+\d+[A-Z]+)'
    match = re.search(pattern, model_name)

    if match:
        return match.group(1)

    # 패턴이 맞지 않으면 전체 모델명 반환
    return model_name

def create_classifier_output(matched_models):
    """매칭된 모델들을 새로운 폴더 구조로 복사"""

    # 출력 폴더 생성
    output_base = Path("classifier_output")
    pdf_output = output_base / "samsung_manuals_classifier_output"
    img_output = output_base / "samsung_img_classifier_output"

    # 폴더 생성
    pdf_output.mkdir(parents=True, exist_ok=True)
    img_output.mkdir(parents=True, exist_ok=True)

    print(f"\n출력 폴더 생성 완료:")
    print(f"- {pdf_output}")
    print(f"- {img_output}")

    # 파일 복사
    copied_count = 0

    for pdf_file, img_folder in matched_models:
        try:
            # PDF 파일 복사
            src_pdf = Path("../pdf_downloader/samsung_manuals") / pdf_file
            dst_pdf = pdf_output / pdf_file

            if src_pdf.exists():
                shutil.copy2(src_pdf, dst_pdf)
                print(f"PDF 복사: {pdf_file}")

            # 이미지 폴더 복사
            src_img = Path("../image_downloader/image_downloader/samsung_img") / img_folder
            dst_img = img_output / img_folder

            if src_img.exists():
                shutil.copytree(src_img, dst_img, dirs_exist_ok=True)
                print(f"이미지 폴더 복사: {img_folder}")

            copied_count += 1

        except Exception as e:
            print(f"복사 중 오류 발생 ({pdf_file}, {img_folder}): {e}")

    print(f"\n총 {copied_count}개 모델의 데이터가 복사되었습니다.")

def main():
    """메인 실행 함수"""
    print("Samsung 세탁기/건조기 모델 분류기 시작")
    print("=" * 50)

    # 현재 디렉토리 확인
    current_dir = Path.cwd()
    print(f"현재 작업 디렉토리: {current_dir}")

    # 필요한 폴더가 존재하는지 확인
    required_dirs = ["../pdf_downloader/samsung_manuals", "../image_downloader/image_downloader/samsung_img"]
    missing_dirs = []

    for dir_name in required_dirs:
        if not Path(dir_name).exists():
            missing_dirs.append(dir_name)

    if missing_dirs:
        print(f"❌ 다음 폴더들이 없습니다: {', '.join(missing_dirs)}")
        print("필요한 폴더들을 확인하고 다시 실행해주세요.")
        return

    # 매칭되는 모델 찾기
    print("\n📋 PDF와 이미지가 모두 있는 모델 찾는 중...")
    matched_models = find_matching_models()

    if not matched_models:
        print("❌ 매칭되는 모델을 찾을 수 없습니다.")
        return

    print(f"\n✅ 총 {len(matched_models)}개의 매칭되는 모델을 발견했습니다.")

    # 사용자 확인
    print("\n매칭된 모델 목록:")
    for i, (pdf_file, img_folder) in enumerate(matched_models, 1):
        print(f"{i:2d}. PDF: {pdf_file}")
        print(f"    IMG: {img_folder}")

    response = input(f"\n이 {len(matched_models)}개 모델을 classifier_output 폴더로 복사하시겠습니까? (y/n): ")

    if response.lower() in ['y', 'yes', '예', 'ㅇ']:
        print("\n📁 파일 복사 시작...")
        create_classifier_output(matched_models)
        print("\n✅ 작업 완료!")
    else:
        print("❌ 작업이 취소되었습니다.")

if __name__ == "__main__":
    main()

Samsung 세탁기/건조기 모델 분류기 시작
현재 작업 디렉토리: C:\Workspaces\SKN14-3rd-3Team\webscraping\YooYonghwan\model_classifier

📋 PDF와 이미지가 모두 있는 모델 찾는 중...
발견된 PDF 파일: 27개
발견된 이미지 폴더: 2개
매칭 발견: Bespoke_AI_콤보_2515kg_1092mm_LCD_WD80F25CHY.pdf <-> Bespoke_AI_콤보_2515kg_1092mm_LCD_WD80F25CHY
매칭 발견: Bespoke_AI_콤보_2518kg_1778mm_LCD_WD90F25AHT.pdf <-> Bespoke_AI_콤보_2518kg_1778mm_LCD_WD90F25AHT

✅ 총 2개의 매칭되는 모델을 발견했습니다.

매칭된 모델 목록:
 1. PDF: Bespoke_AI_콤보_2515kg_1092mm_LCD_WD80F25CHY.pdf
    IMG: Bespoke_AI_콤보_2515kg_1092mm_LCD_WD80F25CHY
 2. PDF: Bespoke_AI_콤보_2518kg_1778mm_LCD_WD90F25AHT.pdf
    IMG: Bespoke_AI_콤보_2518kg_1778mm_LCD_WD90F25AHT

📁 파일 복사 시작...

출력 폴더 생성 완료:
- classifier_output\samsung_manuals_classifier_output
- classifier_output\samsung_img_classifier_output
PDF 복사: Bespoke_AI_콤보_2515kg_1092mm_LCD_WD80F25CHY.pdf
이미지 폴더 복사: Bespoke_AI_콤보_2515kg_1092mm_LCD_WD80F25CHY
PDF 복사: Bespoke_AI_콤보_2518kg_1778mm_LCD_WD90F25AHT.pdf
이미지 폴더 복사: Bespoke_AI_콤보_2518kg_1778mm_LCD_WD90F25AHT

총 2개 모델의 데이터가 복사되었습니다.